In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os

data_path = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

TEST_PATH = os.path.join(data_path, "test.csv")
VAL_PATH = os.path.join(data_path, "validation.csv")
TRAIN_PATH = os.path.join(data_path, "jigsaw-toxic-comment-train.csv")
TRAIN_UIN_PATH = os.path.join(data_path, "jigsaw-toxic-comment-train-processed-seqlen128.csv")

val_data = pd.read_csv(VAL_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)
train_uin_data = pd.read_csv(TRAIN_UIN_PATH)

sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
print(train_uin_data.head())

In [ ]:
#Clean Text
import re

def clean(text):
    text = text.fillna("fillna").str.lower()
    #replace newline characters with space
    text = text.map(lambda x: re.sub('\\n',' ', str(x)))
    text = text.map(lambda x: re.sub('\[\[User.*', '', str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

val_data["comment_text"] = clean(val_data["comment_text"])
test_data["content"] = clean(test_data["content"])
train_data["comment_text"] = clean(train_data["comment_text"]) 

word_vec_length = min(train_data['comment_text'].apply(len).max(), 100) # Length of the input vector
char_vec_length = 100 # Length of the character vector

In [ ]:
#print(train_uin_data.head())
encoded = np.array([train_uin_data['input_word_ids'], train_uin_data['input_mask']])
y_enc = train_uin_data['toxic']
encoded= np.reshape(encoded,(encoded.shape[0], 1, encoded.shape[1]))
print(encoded.shape)
#print(len(train_uin_data['input_word_ids'][3]))

In [ ]:
def build_lstm():
    # Build the model
    print('Build model...')
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(32, return_sequences=False))
    #model.add(Dropout(0.2))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.LSTM(32, return_sequences=True))
    model.add(tf.keras.layers.Dense(units=2, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [ ]:
model = build_lstm()
#model.summary()
batch_size=1000

In [ ]:
model.fit(encoded, y_enc, batch_size=batch_size, epochs=10, validation_data=(val_data['comment_text'], val_data['toxic']))